In [ ]:
import numpy as np
import pandas as pd
import h5py
# Activate auto reload 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%matplotlib ipympl
    
from pyccapt.calibration.data_tools import data_tools

In [ ]:
filename = 'D:/pyccapt/tests/data/data_1642_Aug-30-2023_16-05_Al_test4/data_processing/data_1642_Aug-30-2023_16-05_Al_test4.h5'
df = data_tools.load_data(filename, data_type='pyccapt', mode='processed')
df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


# Plotting the histogram
mc = df['mc_c (Da)'].to_numpy()
mc = mc[mc < 50]
# mc = mc[mc > 26]
fig, ax = plt.subplots()
bin_width = 0.1
bins = np.linspace(np.min(mc), np.max(mc), round(np.max(mc) / bin_width))
y, x, patches = ax.hist(mc, bins)
# Adding labels and title
plt.xlabel('mc (Da)')
plt.ylabel('Frequency')
ax.set_yscale('log')

# Show the plot
plt.show()

In [ ]:
import pybaselines
bkg_1 = pybaselines.polynomial.modpoly(y, x[:-1], poly_order=4)[0]
bkg_2 = pybaselines.whittaker.asls(y, lam=1e7, p=0.02)[0]
bkg_3 = pybaselines.morphological.mor(y, half_window=30)[0]
bkg_4 = pybaselines.smooth.snip(y, max_half_window=40, decreasing=True, smooth_half_window=3)[0]

fig, ax = plt.subplots()
bin_width = 0.1
bins = np.linspace(np.min(mc), np.max(mc), round(np.max(mc) / bin_width))
y, x, patches = ax.hist(mc, bins)
# Adding labels and title
plt.xlabel('mc (Da)')
plt.ylabel('Frequency')
ax.set_yscale('log')

plt.plot(x[:-1], bkg_1, '--', label='modpoly')
plt.plot(x[:-1], bkg_2, '--', label='asls')
plt.plot(x[:-1], bkg_3, '--', label='mor')
plt.plot(x[:-1], bkg_4, '--', label='snip')

plt.legend()
plt.show()

In [ ]:
half_window_1 = 15
half_window_2 = 45
fit_1, params_1 = baseline_fitter.std_distribution(y, half_window_1, smooth_half_window=10)
fit_2, params_2 = baseline_fitter.std_distribution(y, half_window_2, smooth_half_window=10)

fig, ax = plt.subplots()
bin_width = 0.1
bins = np.linspace(np.min(mc), np.max(mc), round(np.max(mc) / bin_width))
y, x, patches = ax.hist(mc, bins)
# Adding labels and title
plt.xlabel('mc (Da)')
plt.ylabel('Frequency')
ax.set_yscale('log')
plt.plot(x, fit_1, label=f'half_window={half_window_1}')
plt.plot(x, fit_2, '--', label=f'half_window={half_window_2}')
plt.legend()
plt.show()

In [ ]:
from skimage.filters import threshold_triangle as custom_threshold
from pybaselines import Baseline
fig, ax = plt.subplots()
bin_width = 0.1
bins = np.linspace(np.min(mc), np.max(mc), round(np.max(mc) / bin_width))
y, x, patches = ax.hist(mc, bins)
# Adding labels and title
plt.xlabel('mc (Da)')
plt.ylabel('Frequency')
ax.set_yscale('log')

half_window_1 = 30
half_window_2 = 15

baseline_fitter = Baseline(x_data=x[:-1])
fit_1, params_1 = baseline_fitter.fastchrom(y, half_window_2)
fit_2, params_2 = baseline_fitter.fastchrom(y, half_window_1, threshold=custom_threshold)


# fit_3, params_3 = pybaselines.classification.cwt_br(y)
fit_4, params_4 = pybaselines.classification.dietrich(y)
fit_5, params_5 = pybaselines.classification.fabc(y, lam=1e6, num_std=3.0)
fit_6, params_6 = pybaselines.classification.fastchrom(y)
fit_7, params_7 = pybaselines.classification.golotvin(y)
fit_8, params_8 = pybaselines.classification.std_distribution(y)



plt.plot(x[:-1], fit_1, label='default')
plt.plot(x[:-1], fit_2, label='aspls', color='red')
plt.plot(x[:-1], fit_3, ':', label='cwt_br')
# plt.plot(x[:-1], fit_4, ':', label='dietrich')
plt.plot(x[:-1], fit_5, ':', label='fabc')
plt.plot(x[:-1], fit_6, ':', label='fastchrom')
plt.plot(x[:-1], fit_7, ':', label='golotvin')
plt.plot(x[:-1], fit_8, ':', label='std_distribution')
plt.legend()
plt.show()

In [ ]:
# GMM number of componnents
from sklearn import mixture

tt = np.expand_dims(mc, axis=1)
tt = np.float32(tt)

mask = np.random.randint(0, len(tt), 10000)
tt = tt[mask]

n_components = np.arange(1, 4)
models = [mixture.GaussianMixture(n, covariance_type='full', random_state=0).fit(tt)
          for n in n_components]

fig1, ax1 = plt.subplots(figsize=(6, 6))
bic_test = [m.bic(tt) for m in models]
aic_test = [m.aic(tt) for m in models]
plt.plot(n_components, bic_test, label='BIC')
plt.plot(n_components, aic_test, label='AIC')
plt.legend(loc='best')
plt.xlabel('n_components', color='r')
plt.grid(color='aqua', alpha=0.3, linestyle='-.', linewidth=2)
plt.show()
n_components = min(np.argmin(bic_test), np.argmin(aic_test))
print('Number of components is:', n_components)

In [ ]:
from sklearn.mixture import GaussianMixture


gmm = GaussianMixture(n_components=3, covariance_type='diag').fit(tt)

mc_g = np.expand_dims(mc, axis=1)
mc_g = np.float32(mc_g)
labels = gmm.predict(mc_g)



fig1, ax1 = plt.subplots(figsize=(8, 6))
bins = 0.1
bins = np.linspace(np.min(mc_g), np.max(mc_g), round(np.max(mc_g) / bins))
plt.hist(mc, bins, log=True)
fig1, ax1 = plt.subplots(figsize=(8, 6))

for i in range(n_components):
    A = mc_g[labels==i]
    plt.hist(A, bins, log=True)
plt.show()

In [ ]:
from sklearn.cluster import MeanShift
import matplotlib.pyplot as plt
import numpy as np

# Assuming tt is your data, mc is your feature, and n_components is the number of clusters
m_shift = MeanShift(bandwidth=0.1).fit(tt)

mc_g = np.expand_dims(mc, axis=1)
mc_g = np.float32(mc_g)
labels = m_shift.predict(mc_g)



fig1, ax1 = plt.subplots(figsize=(8, 6))
bins = 0.1
bins = np.linspace(np.min(mc_g), np.max(mc_g), round(np.max(mc_g) / bins))
plt.hist(mc, bins, log=True)
fig1, ax1 = plt.subplots(figsize=(8, 6))
for i in range(n_components):
    A = mc_g[labels==i]
    plt.hist(A, bins, log=True)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler

n_components=4

mc_g = np.expand_dims(mc, axis=1)
mc_g = np.float32(mc_g)


# Standardize the data
tt = StandardScaler().fit_transform(tt)

# K-Means clustering
kmeans = KMeans(n_clusters=n_components, random_state=42)
kmeans_labels = kmeans.fit(tt)
kmeans_labels = kmeans.predict(mc_g)
num_kmeans_labels = len(np.unique(kmeans_labels))

# Agglomerative Hierarchical clustering
# agg_clustering = AgglomerativeClustering(n_clusters=n_components)
# agg_labels = agg_clustering.fit_predict(mc_g)

# DBSCAN clustering
dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan_labels = dbscan.fit_predict(mc_g)
num_dbscan_labels = len(np.unique(dbscan_labels))

# GMM clustering (as you've already implemented)
gmm = GaussianMixture(n_components=n_components, covariance_type='diag').fit(tt)
gmm_labels = gmm.predict(mc_g)

# Plotting
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# K-Means

axes[0, 0].set_title('K-Means Clustering')
for i in range(num_kmeans_labels):
    A = mc_g[kmeans_labels==i]
    axes[0, 0].hist(A, bins, log=True)
# # Agglomerative Hierarchical
# for i in range(n_components):
#     A = mc_g[agg_labels==i]
#     axes[0, 1].hist(A, bins, log=True)
# axes[0, 1].set_title('Agglomerative Clustering')

# DBSCAN
for i in range(num_dbscan_labels):
    A = mc_g[dbscan==i]
    axes[1, 0].hist(A, bins, log=True)
axes[1, 0].set_title('DBSCAN Clustering')

# GMM
for i in range(n_components):
    A = mc_g[gmm_labels==i]
    axes[1, 1].hist(A, bins, log=True)
axes[1, 1].set_title('GMM Clustering')

plt.tight_layout()
plt.show()
